# Web Scraping Chronicling America OCR

I crafted this program after needing to put together a dataset of newspaper-specific textual data from Chronicling America. I'm sharing it here in case others need to build similar datasets. To use this program, you'll need to identify the newspaper that has been digitized and OCR'ed in Chronicling America. You'll need at least one url for an OCR page of the given newspaper. It will be good to designate a timeframe of printing that you want to extract from the newspaper as well.

This program utilizes patterns in Chronicling America's urls. For example, consider this page:

https://chroniclingamerica.loc.gov/lccn/sn84024736/1814-09-28/ed-1/seq-1/ocr/

You'll notice the url contains metadata. For example, 'sn84024736' is the Library of Congress's code for the given newspaper, the Inquirer, an early nineteenth century newspaper from Richmond, VA. Then there's the date: 1814-09-28. This tells you this url directs to the newspaper on that date. 'ed-1' refers to the edition scanned. 'seq-1' stands for sequence 1, or the first page of the newspaper on that day.

With this information, I've been able to write this program which iterates over small changes to the url, extracts text from each combination of the url, and then removes any "Not Found" or dead links. The remaining output is a dataframe containing only the text for the given newspaper over the course of a predetermined period.

Before running this program, please consider the following:

- It can take a long time. Iterating over so many combinations of the url can take hours, or even days, depending on the range of dates you're web scraping. Given this limitation, you'll want to reduce the date range as much as possible. A shorter range equals fewer iterations, and subsequently less time and computer power needed.
- Given the long timeframe this program could potentially take, be sure to deactivate sleep mode on your device. You need to be able to maintain VPN connection for this program to work.
- This program is highly adaptable. The current version is aimed at extracting print periods for individual newspapers, but there are a number of ways of rewriting this code to fit other web scraping needs.
- Double-check your url indices. Simply running this code without ensuring it is tailored to your specific url will be a waste of time and effort.
- Further data preprocessing will probably be needed. This program only takes the html-formatted text from the urls and puts it into a dataframe. From there, you can munge the data in all sorts of ways. Just keep in mind this version of the program is limited to building a preliminary dataset, not a clean or processed one.

### Import Libraries

In [1]:
import pandas as pd
import requests

### Define your row-entry function, your dataframe, and your url.

These preliminary steps will need to be double-checked within the rest of the program; that is, any changes to the demonstration versions below will also need to be changed throughout the program.

In [2]:
def insert_row(data, my_row):
    data.loc[len(data)] = my_row

df = pd.DataFrame(columns = ['url', 'text', 'date', 'not_found'])
url_string = 'https://chroniclingamerica.loc.gov/lccn/sn84024736/1814-09-28/ed-1/seq-1/ocr/'

### Define Iteration Functions

The following functions iterate over characters in url_string based on their indices. To identify the indices of a given character, you need to count the characters from left to right. So, the first character is 'h'. Its index is 0. The date indices in url_string are 51 to 60. You can double-check your indices with this line of code:

In [8]:
url_string[0]

'h'

In [9]:
url_string[51:61]

'1814-09-28'

Keep these indices in mind. To manipulate the program to extract different url pages, you'll need to adjust the following programs by changing their inputted indices.

In [10]:
def change_decade(string):
    for i in range(0, 10): # Change range for preferred decade range. 0, 10 runs through all decades in the century.
        # If you only want the first decade in the century, use range(0, 1). And so on.
        new_character = str(i)
        string_list = list(string)
        index_to_change = 53
        string_list[index_to_change] = new_character
        modified_string = ''.join(string_list)
        # print(modified_string) # add if you want to see the progress as the program runs
        url = requests.get(modified_string)
        text = url.text
        insert_row(df, [modified_string, text, 'NaN', 'NaN'])

In [11]:
def change_year(string):
    for i in range(4, 6): # Change range for preferred year range. Do you only want years 1814-1815, for example?
        # If so, range(4, 6) is correct. Do you want years 1810 to 1819? range(0,10) would be correct. And so on.
        new_character = str(i)
        string_list = list(string)
        index_to_change = 54
        string_list[index_to_change] = new_character
        modified_string = ''.join(string_list)
        # change_decade(modified_string) # Add this if you want to attempt scraping by decade.
        # Note: this could take a very long time!
        # print(modified_string) # add if you want to see the progress as the program runs
        url = requests.get(modified_string)
        text = url.text
        insert_row(df, [modified_string, text, 'NaN', 'NaN'])

In [12]:
# This function changes the first number in the month.
def change_month(string):
    for i in range(0, 2): # The first number in a month can only be 0 or 1 (i.e. july is 07, November is 11)
        new_character = str(i)
        string_list = list(string)
        index_to_change = 56
        string_list[index_to_change] = new_character
        modified_string = ''.join(string_list)
        change_year(modified_string) # Add this if you want to scrape by month.
        # print(modified_string) # add if you want to see the progress as the program runs
        url = requests.get(modified_string)
        text = url.text
        insert_row(df, [modified_string, text, 'NaN', 'NaN'])

In [13]:
# This function changes the second number in the month.
def change_month2(string):
    for i in range(0, 10): # Second numbers in the month can be anything from 1 to 9.
        new_character = str(i)
        string_list = list(string)
        index_to_change = 57
        string_list[index_to_change] = new_character
        modified_string = ''.join(string_list)
        change_month(modified_string) # Add this if you want to scrape by month.
        # print(modified_string) # add if you want to see the progress as the program runs
        url = requests.get(modified_string)
        text = url.text
        insert_row(df, [modified_string, text, 'NaN', 'NaN'])

In [14]:
# This function changes the first number in the day.
def change_day(string):
    for i in range(0, 4): # This range can only be 0 to 3, as in "01 is the first day of the month, 31 is the last".
        new_character = str(i)
        string_list = list(string)
        index_to_change = 59
        string_list[index_to_change] = new_character
        modified_string = ''.join(string_list)
        change_month2(modified_string) # Add this if you want to scrape by day.
        # print(modified_string) # add if you want to see the progress as the program runs
        url = requests.get(modified_string)
        text = url.text
        insert_row(df, [modified_string, text, 'NaN', 'NaN'])

In [15]:
# This function changes the second number in the day.
def change_day2(string):
    for i in range(0, 10): # This range can be any number from 1 to 9.
        new_character = str(i)
        string_list = list(string)
        index_to_change = 60
        string_list[index_to_change] = new_character
        modified_string = ''.join(string_list)
        change_day(modified_string) # Add this if you want to scrape by day.
        # print(modified_string) # add if you want to see the progress as the program runs
        url = requests.get(modified_string)
        text = url.text
        insert_row(df, [modified_string, text, 'NaN', 'NaN'])

In [16]:
def change_page(string):
    for i in range(0, 5): # Change this range to fit the number of digitized pages per issue of given newspaper.
        # You'll need to look this up on Chronicling America.
        new_character = str(i)
        string_list = list(string)
        index_to_change = 71
        string_list[index_to_change] = new_character
        modified_string = ''.join(string_list)
        change_day2(modified_string) # Add this if you want to scrape by the other functions.
        # print(modified_string) # add if you want to see the progress as the program runs
        url = requests.get(modified_string)
        text = url.text
        insert_row(df, [modified_string, text, 'NaN', 'NaN'])

### Run the program

Have you double-checked all the ranges, indices, and added or removed functions? If so, you can run the program:

In [17]:
# change_page(url_string)

Now you wait. Eventually, you'll have a complete dataframe saved as df that contains all the urls and their text. But you're not done yet. df also contains text from "Not Found" urls or broken links. To filter this stuff out, try this:

In [18]:
df['date'] = df['url'].str.findall(r'\d\d\d\d-\d\d-\d\d')
df['not_found'] = df['text'].str.findall('dublincore.org')
df['not_found'] = df['not_found'].astype(str)
substring = 'dublincore.org'
filter = df['not_found'].str.contains(substring)
df = df[~filter]
df = df.drop_duplicates(subset=['url'])
df = df.sort_values('date')
df.head()

,url,text,date,not_found


This block of code fills the date column with the combination from the url. It finds strings of 'dublincore.org'–text that only appears on "Not Found" or broken link pages, and then filters those instances out of df. It also sorts df by date and drops any duplicates. The result should be a dataframe with only the text in html format from every Chronicling America url in your chosen ranges. Hooray!

In [19]:
df

,url,text,date,not_found


Need to save your data as a csv file? Try changing the file name then running this line of code:

In [ ]:
df.to_csv('inquirer_1814-15.csv', index=False, encoding='utf-8')